<a href="https://colab.research.google.com/github/hanhpm/Project-RAG-PDFvQA/blob/main/rag_program.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installing necessary library packages
! pip install -q transformers==4.41.2
! pip install -q bitsandbytes==0.43.1
! pip install -q accelerate==0.31.0
! pip install -q langchain==0.2.5
! pip install -q langchainhub==0.1.20
! pip install -q langchain-chroma==0.1.1
! pip install -q langchain-community==0.2.5
! pip install -q langchain_huggingface==0.0.3
! pip install -q python-dotenv==1.0.1
! pip install -q pypdf==4.2.0
! pip install -q numpy==1.25.0

In [ ]:
# Build vector database
# Sub-step: Import necessary libraries
import torch

from transformers import BitsAndBytesConfig
from transformers import AutoTokenizer , AutoModelForCausalLM , pipeline
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_huggingface.llms import HuggingFacePipeline

from langchain.memory import ConversationBufferMemory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import PyPDFLoader , TextLoader
from langchain.chains import ConversationalRetrievalChain

from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub

In [ ]:
# Sub-step: Reading file pdf
# Initialize class PyPDFLoader
Loader = PyPDFLoader
FILE_PATH = "/content/YOLOv10_Tutorials.pdf" # Default path: /content/{name file}
loader = Loader(FILE_PATH)
documents = loader.load()

In [ ]:
# Sub-step: Initialize class Text Splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = text_splitter

# Function break down file pdf
docs = text_splitter.split_documents(documents)

print("Number of sub-documents: ", len(docs))
print(docs[0])

Number of sub-documents:  33
page_content='AI VIET NAM – AI COURSE 2024\nTutorial: Phát hiện đối tượng trong ảnh với\nYOLOv10\nDinh-Thang Duong, Nguyen-Thuan Duong, Minh-Duc Bui và\nQuang-Vinh Dinh\nNgày 20 tháng 6 năm 2024\nI. Giới thiệu\nObject Detection (Tạm dịch: Phát hiện đối tượng) là một bài toán cổ điển thuộc lĩnh vực\nComputer Vision. Mục tiêu của bài toán này là tự động xác định vị trí của các đối tượng trong\nmột tấm ảnh. Tính tới thời điểm hiện tại, đã có rất nhiều phương pháp được phát triển nhằm\ngiải quyết hiệu quả bài toán này. Trong đó, các phương pháp thuộc họ YOLO (You Only Look\nOnce) thu hút được sự chú ý rất lớn từ cộng đồng nghiên cứu bởi độ chính xác và tốc độ thực\nthi mà loại mô hình này mang lại.\nHình 1: Logo của mô hình YOLO. Ảnh: link.\nThời gian vừa qua, Ao Wang và các cộng sự tại Đại học Thanh Hoa (Tsinghua University)\nđã đề xuất mô hình YOLOv10 trong bài báo YOLOv10: Real-Time End-to-End Object\nDetection [10]. Với những cải tiến mới, mô hình đã đạt đư

In [ ]:
# Sub-step: Initialize instance vectorization:
# Vì các văn bản gốc biểu diễn dưới dạng str nên gây khó khăn cho truy vấn dẫn đến output 0 tốt
# Embedding instance help to convert text to vector
embeddings = HuggingFaceEmbeddings()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
# Sub-step: Initialize vector database
vector_db = Chroma.from_documents(documents=docs, embedding=embeddings)
retriever = vector_db.as_retriever()
result = retriever.invoke("What is YOLOv10?")
print("Number of revelant documents: ", len(result))

Number of revelant documents:  4


In [ ]:
# Initialize model LLMs: model Vicuna(good response with Vietnamese)
# Sub-step: Declare some necessary settings for the model
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
# Sub-step: Initialize model and tokenizer
MODEL_NAME = "lmsys/vicuna-7b-v1.5"

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=nf4_config,
    low_cpu_mem_usage=True
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

In [ ]:
# Sub-step: Integrate tokenizer and model into one pipeline for ease of use
model_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    pad_token_id=tokenizer.eos_token_id,
    device_map="auto",
)

llm = HuggingFacePipeline(pipeline=model_pipeline)

In [ ]:
# Run program
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
  return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

USER_QUESTION = "Yolov10 là gì?"
output = rag_chain.invoke(USER_QUESTION)
answer = output.split('Answer: ')[1].strip()
print(answer)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Yolov10 là một phiên bản của YOLO (You Only Look Once) - một hệ thống dự đoán hình ảnh được huấn luyện sẵn trên bộ dữ liệu COCO. Phiên bản Yolov10 được cài đặt và sử dụng trên Google Colab bằng cách tải về trọng số của pre-trained model và khởi tạo mô hình từ trọng số đó.
